In [1]:
import pandas as pd
import plotnine as p9
import numpy as np
import matplotlib
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'sans-serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})
df = pd.read_csv("microbenchmark.csv")
base_theme = (p9.theme_classic(base_size=14, base_family='sans-serif')
              + p9.theme(axis_text_y=p9.element_text(margin={'r': 5}),
                         legend_box_spacing=0.01,
                         legend_box_margin=0,
                         legend_margin=0,
                         legend_position='top',
                         figure_size=(10,5),
                         text=p9.element_text(size=14)))

In [2]:
def scheme(mode, *colors):
    colordef = {'lred': '#fab7b7', 
                'dred': '#d45079',
                'dblue': '#6886c5', 
                'lblue': '#b0deff', 
                'dorange': '#f5b971',
                'lorange': '#f6d186',
                'dpurp': '#a6b1e1',
                'yellow': '#fcf876',
                'lpurp': '#dcd6f7',
                'lgreen': '#a8e6cf',
                'blackish': '#39375b',
                'seafoam': '#a0ffe6'}
    scfun = p9.scale_fill_manual if mode == 'fill' else p9.scale_color_manual
    if bool(colors):
        return scfun([(colordef[x] if x in colordef else x) for x in colors])
    else:
        return scfun(list(colordef.values()))

In [3]:
def cycles_for(df, op):
    return (df[df.operation == op]
            .groupby(['fs', 'size'])
            .median().apply(np.ceil)
            .cycles.reset_index(name="cycles"))

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [12]:
(p9.ggplot(df.pipe(cycles_for, 'read')) 
 + p9.aes(x='size', y='cycles', color='fs') 
 + p9.geom_line(size=1, alpha=0.7)
 + p9.geom_point()
 + base_theme
 + scheme('color', 'dblue', 'dred')
 + p9.theme(figure_size=(10,5))
 + p9.labs(x='Size (Bytes)', y='CPU cycles', color='Implementation')).save("plots/read.png", dpi=300)

In [11]:
(p9.ggplot(df.pipe(cycles_for, 'write')) 
 + p9.aes(x='size', y='cycles', color='fs') 
 + p9.geom_line(size=1, alpha=0.7)
 + p9.geom_point()
 + base_theme
 + scheme('color', 'dblue', 'dred')
 + p9.theme(figure_size=(10,5))
 + p9.labs(x='Size (Bytes)', y='CPU cycles', color='Implementation')).save("plots/write.png", dpi=300)

In [7]:
for idx, row in df.groupby(['fs', 'operation', 'size']).median().apply(np.ceil).cycles.reset_index(name='cycles').iterrows():
    if row['size'] < 0:
        fname = f"stats/{row.fs}-{row.operation}.tex"
    else:
        fname = f'stats/{row.fs}-{row.operation}-{row["size"]}.tex'
        
    with open(fname, "w") as f:
        f.write(str(int(row.cycles)))


In [10]:
medians = df.groupby(['fs', 'operation', 'size']).median().apply(np.ceil).cycles.reset_index(name='cycles')
pd.DataFrame({'fs': medians.fs,
              'op': medians.apply(lambda x: x.operation if x['size'] < 0 else ' '.join([x.operation, str(x['size'])]), axis='columns'),
              'cycles': medians.cycles.map(int)}).pivot(index='fs', columns='op', values='cycles').to_latex("test.tex")